Check npy format

In [ ]:
import numpy as np
file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/X_arm.npy"
data = np.load(file_path)

print(data.shape)

print(data[:5])

In [ ]:
mips_data = np.load("/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/X_mips.npy")
print(mips_data.shape)

In [ ]:
label = np.load("/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/y.npy")

print(label.shape)
print(label[:10])
print(label[40000:40010])

In [ ]:
import pickle 

file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/word2vec/CBOW/sentences_train.pkl"

with open(file_path, 'rb') as f:
    sentences = pickle.load(f)

print(len(sentences))

Check npy similarity

In [10]:
import torch
import numpy as np

def similarity_score(x_arm, x_mapped):
    # Eq.(7): s = 1 / (1 + ||arm - mapped||_2)
    dist = torch.norm(x_arm - x_mapped, p=2, dim=1)  # [B]
    return 1.0 / (1.0 + dist)


arm_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/X_arm.npy"
x86_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/X_x86.npy"
label_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/y.npy"

arm_np = np.load(arm_path)
x86_np = np.load(x86_path)
label_np = np.load(label_path)

#check npy contents

print("ARM shape:", arm_np.shape)
print("x86 shape:", x86_np.shape)
print("Label shape:", label_np.shape)


print("ARM first 5 vectors:", arm_np[:5])
print("x86 first 5 vectors:", x86_np[:5])
print("Labels first 10:", label_np[:10])



ARM shape: (22119, 256)
x86 shape: (22119, 256)
Label shape: (22119,)
ARM first 5 vectors: [[ 8.2690967e-04 -1.0663398e-03 -7.6523237e-04 ...  6.4992113e-05
   8.1985537e-04  6.0439383e-05]
 [ 2.5329064e-03 -2.1999115e-03  5.1784003e-04 ... -2.4773190e-03
   3.9047841e-03 -3.3722823e-03]
 [-6.9914960e-05 -1.3632157e-03 -6.8722075e-05 ... -5.9428352e-05
   9.6438429e-04  4.3566377e-04]
 [ 2.5329064e-03 -2.1999115e-03  5.1784003e-04 ... -2.4773190e-03
   3.9047841e-03 -3.3722823e-03]
 [ 2.0768049e-03 -1.0403584e-03 -6.7534682e-04 ... -1.3996513e-03
   2.9023981e-04 -1.3352833e-03]]
x86 first 5 vectors: [[ 2.5923559e-04 -1.4290018e-03 -9.3509363e-05 ...  5.9786270e-04
   6.2040339e-04 -2.5504397e-04]
 [ 2.5329064e-03 -2.1999115e-03  5.1784003e-04 ... -2.4773190e-03
   3.9047841e-03 -3.3722823e-03]
 [-5.5855548e-04  9.7668730e-05  1.3857879e-03 ...  2.8943561e-04
   1.4697339e-03 -1.6440039e-03]
 [ 2.2104110e-03  2.3895921e-04  2.0589645e-03 ... -2.7211525e-03
   2.8053937e-03 -3.4100977e-

In [ ]:
import random
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# 路徑
arm_csv  = "/home/tommy/Projects/pcodeFcg/dataset/alignment/arm_32_alignment.csv"
mips_csv = "/home/tommy/Projects/pcodeFcg/dataset/alignment/mips_32_alignment.csv"
w2v_path  = "/home/tommy/Projects/pcodeFcg/vector/contrastive/word2vec/CBOW/word2vec.model"

# 讀檔
df_arm  = pd.read_csv(arm_csv)   # 欄位：file_name,function_name,pcode,label
df_mips = pd.read_csv(mips_csv)
w2v     = Word2Vec.load(w2v_path)

# 前 5 筆同列比對（ARM vs MIPS row-wise）
print("=== 同一列前 5 筆 Cosine Similarity ===")
for i in range(5):
    toks_arm  = str(df_arm.loc[i, "pcode"]).split()
    toks_mips = str(df_mips.loc[i, "pcode"]).split()
    vec_arm   = np.mean([w2v.wv[t] for t in toks_arm  if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    vec_mips  = np.mean([w2v.wv[t] for t in toks_mips if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    sim        = cosine_similarity(vec_arm.reshape(1,-1), vec_mips.reshape(1,-1))[0,0]
    print(f"Row {i}: {sim:.4f}")

# 隨機 5 筆不同行比對（ARM row i vs MIPS row j, i != j）
print("\n=== 隨機 5 筆不同行 Cosine Similarity ===")
n = len(df_arm)
pairs = set()
while len(pairs) < 5:
    i = random.randrange(n)
    j = random.randrange(n)
    if i != j:
        pairs.add((i, j))

for i, j in pairs:
    toks_arm  = str(df_arm.loc[i, "pcode"]).split()
    toks_mips = str(df_mips.loc[j, "pcode"]).split()
    vec_arm   = np.mean([w2v.wv[t] for t in toks_arm  if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    vec_mips  = np.mean([w2v.wv[t] for t in toks_mips if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    sim        = cosine_similarity(vec_arm.reshape(1,-1), vec_mips.reshape(1,-1))[0,0]
    print(f"ARM row {i} vs MIPS row {j}: {sim:.4f}")


In [ ]:
import os
import pickle
import torch
import networkx as nx

# 1. 載入你訓練好的 Adapter
from model import Adapter  
ckpt = torch.load(
    '/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/simsiam_adapter.pth',
    map_location='cpu'
)
state_dict = ckpt.get('state_dict', ckpt)
adapter_state = {
    k[len('backbone.'):]: v
    for k, v in state_dict.items()
    if k.startswith('backbone.adapter.')
}
adapter = Adapter(dim=256, num_blocks=2)
adapter.load_state_dict(adapter_state, strict=False)
adapter.eval()

# 2. 指定單一對應檔案路徑
path_src = '/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/train/0b/0b61ed19ec8d9450268020524aae18ff68071164042097c346afe9085b72135d.gpickle'
path_tgt = '/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/test/0a/0a3eab21fff9e3edbb22861e6d1c461dee66e36cc2f0010602eaff0496ed8271.gpickle'

# 3. 讀入圖
with open(path_src, 'rb') as f:
    G_src = pickle.load(f)
    print(G_src)
    for node in G_src.nodes():
        print(G_src.nodes[node]['vector'])
with open(path_tgt, 'rb') as f:
    G_tgt = pickle.load(f)
    print(G_tgt)

# 4. 比較每個 node
print(f'Comparing {os.path.basename(path_src)} → {os.path.basename(path_tgt)}\n')
print('Node\tOrigDist\tMappedDist')
for node in G_src.nodes():
    if node not in G_tgt:
        continue
    x_src = torch.tensor(G_src.nodes[node]['vector'], dtype=torch.float32)
    x_tgt = torch.tensor(G_tgt.nodes[node]['vector'], dtype=torch.float32)
    # 原始距離
    d_orig = torch.dist(x_src, x_tgt).item()
    # 經 adapter 映射後距離
    x_mapped = adapter(x_src.unsqueeze(0)).squeeze(0)
    d_mapped = torch.dist(x_mapped, x_tgt).item()
    print(f'{node}\t{d_orig:.4f}\t{d_mapped:.4f}')


In [3]:
import pickle

file_path = "/home/tommy/Projects/pcodeFcg/dataset/alignment/train.pickle"

with open(file_path, 'rb') as f:
    data = pickle.load(f)

print(f"Data loaded from {file_path}, length: {len(data)}")
print(data[:1])  # Print first 5 items to verify structure


Data loaded from /home/tommy/Projects/pcodeFcg/dataset/alignment/train.pickle, length: 22119
[('INT_NOTEQUAL-REG-MEM-CONST CBRANCH-MEM-REG CALL-MEM INDIRECT-MEM-MEM-CONST INDIRECT-MEM-MEM-CONST COPY-MEM-CONST RETURN-CONST COPY-MEM-MEM COPY-MEM-MEM RETURN-CONST COPY-MEM-MEM COPY-MEM-MEM', 'COPY-UNIQUE-MEM LOAD-UNIQUE-CONST-MEM INT_NOTEQUAL-REG-UNIQUE-CONST CBRANCH-MEM-REG RETURN-CONST COPY-MEM-MEM CALL-MEM INDIRECT-MEM-MEM-CONST STORE-CONST-UNIQUE-CONST INDIRECT-MEM-MEM-CONST RETURN-CONST COPY-MEM-MEM', 1)]
